## kNN: Выбор числа соседей

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import scale

#### Данное задание основано на материалах лекций по метрическим методам и посвящено подбору числа соседей в методе kNN.

Метрические методы основаны на гипотезе компактности, суть которой состоит в том, что объекты с похожими признаковыми описаниями имеют похожие значения целевой переменной. Если эта гипотеза верна, то строить прогноз для нового объекта можно на основе близких к нему объектов из обучающей выборки — например, путем усреднения их ответов (для регрессии) или путем выбора наиболее популярного среди них класса (для классификации). Методы такого типа и называются метрическими. Они имеют несколько особенностей:

- Процедура обучения, по сути, отсутствует — достаточно лишь запомнить все объекты обучающей выборки
- Можно использовать метрику, учитывающую особенности конкретного набора данных — например, наличие категориальных (номинальных) признаков
- При правильном выборе метрики и достаточном размере обучающей выборки метрические алгоритмы показывают качество, близкое к оптимальному

Метрические методы чувствительны к масштабу признаков — так, если масштаб одного из признаков существенно превосходит масштабы остальных признаков, то их значения практически не будут влиять на ответы алгоритма. Поэтому важно производить масштабирование признаков. Обычно это делается путем вычитания среднего значения признака и деления на стандартное отклонение.

### Реализация в Scikit-Learn

Метод k ближайших соседей реализован в классе [<code>sklearn.neighbors.KNeighborsClassifier</code>](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html). Основным параметром является n_neighbors, который задает число соседей для построения прогноза.

Вам понадобится производить кросс-валидацию по блокам. Кросс-валидация заключается в разделении выборки на m непересекающихся блоков примерно одинакового размера, после чего выполняется m шагов. На i-м шаге i-й блок выступает в качестве тестовой выборки, объединение всех остальных блоков — в качестве обучающей выборки. Соответственно, на каждом шаге алгоритм обучается на некоторой обучающей выборке, после чего вычисляется его качество на тестовой выборке. После выполнения m шагов мы получаем m показателей качества, усреднение которых и дает оценку кросс-валидации. Подробнее вы можете послушать про кросс-валидацию в видео "Проблема переобучения. Методология решения задач машинного обучения" из первого модуля, а также почитать на [Википедии](https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D0%B5%D0%BA%D1%80%D1%91%D1%81%D1%82%D0%BD%D0%B0%D1%8F_%D0%BF%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0) или в [документации scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html).

Технически кросс-валидация проводится в два этапа:

- Создается генератор разбиений <code>sklearn.model_selection.KFold</code>, который задает набор разбиений на обучение и валидацию. Число блоков в кросс-валидации определяется параметром n_splits. Обратите внимание, что порядок следования объектов в выборке может быть неслучайным, это может привести к смещенности кросс-валидационной оценки. Чтобы устранить такой эффект, объекты выборки случайно перемешивают перед разбиением на блоки. Для перемешивания достаточно передать генератору KFold параметр shuffle=True.
- Вычислить качество на всех разбиениях можно при помощи функции <code>sklearn.model_selection.cross_val_score</code>. В качестве параметра estimator передается классификатор, в качестве параметра cv — генератор разбиений с предыдущего шага. С помощью параметра scoring можно задавать меру качества, по умолчанию в задачах классификации используется доля верных ответов (accuracy). Результатом является массив, значения которого нужно усреднить.

Приведение признаков к одному масштабу можно делать с помощью функции <code>sklearn.preprocessing.scale</code>, которой на вход необходимо подать матрицу признаков и получить масштабированную матрицу, в которой каждый столбец имеет нулевое среднее значение и единичное стандартное отклонение.

В этом задании вам нужно подобрать оптимальное значение k для алгоритма kNN. Будем использовать набор данных Wine, где требуется предсказать сорт винограда, из которого изготовлено вино, используя результаты химических анализов.

In [2]:
def write_answer(filename, answer):
    
    with open(filename, 'w') as fout:
        
        fout.write(str(answer))
        fout.close()

#### 1. Загрузите выборку Wine по адресу https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data

Извлеките из данных признаки и классы. Класс записан в первом столбце (три варианта), признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine 

In [3]:
wine = pd.read_csv('wine.data', header=None)

wine.columns = ['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 
'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
'Proline']

y = wine['Class']
X = wine.iloc[:, 1:]

wine.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


#### 2. Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).

In [4]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

#### 3. Найдите точность классификации на кросс-валидации для метода k ближайших соседей (<code>sklearn.neighbors.KNeighborsClassifier</code>), при k от 1 до 50. При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)? Данные результаты и будут ответами на вопросы 1 и 2.

In [5]:
max_cv_score, k_optim = 0, 0

for k in range(1, 51):
    clf = KNeighborsClassifier(n_neighbors=k)
    cv_score = np.mean(cross_val_score(clf, X, y, cv=kfold))
    
    if cv_score > max_cv_score:
        max_cv_score = cv_score
        k_optim = k

print(f'Optimal number of neighbours of kNN: k = {k_optim}')
print(f'Best cv_accuracy of kNN with k = {k_optim}: acc = {max_cv_score}')

write_answer('submission_wine_1.txt', k_optim)
write_answer('submission_wine_2.txt', round(max_cv_score, 2))

Optimal number of neighbours of kNN: k = 1
Best cv_accuracy of kNN with k = 1: acc = 0.7304761904761905


#### 4. Произведите масштабирование признаков с помощью функции <code>sklearn.preprocessing.scale</code>. Снова найдите оптимальное k на кросс-валидации.

In [6]:
X_scaled = scale(X)

#### 5. Какое значение k получилось оптимальным после приведения признаков к одному масштабу? Приведите ответы на вопросы 3 и 4. Помогло ли масштабирование признаков?

In [7]:
max_cv_score, k_optim = 0, 0

for k in range(1, 51):
    clf = KNeighborsClassifier(n_neighbors=k)
    cv_score = np.mean(cross_val_score(clf, X_scaled, y, cv=kfold))
    
    if cv_score > max_cv_score:
        max_cv_score = cv_score
        k_optim = k

print(f'Optimal number of neighbours of kNN: k = {k_optim}')
print(f'Best cv_accuracy of kNN with k = {k_optim}: acc = {max_cv_score}')

write_answer('submission_wine_3.txt', k_optim)
write_answer('submission_wine_4.txt', round(max_cv_score, 2))

Optimal number of neighbours of kNN: k = 29
Best cv_accuracy of kNN with k = 29: acc = 0.9776190476190475
